In [1]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [2]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
filepath = Path("../Credit_Risk/Resources/lending_data.csv")
df = pd.read_csv(filepath)


# Review the DataFrame
df.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


In [3]:
#adding this cell to look at column data types
df.dtypes

loan_size           float64
interest_rate       float64
borrower_income       int64
debt_to_income      float64
num_of_accounts       int64
derogatory_marks      int64
total_debt            int64
loan_status           int64
dtype: object

### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [4]:
# Separate the data into labels and features
# Separate the y variable, the labels
y = df["loan_status"]


# Separate the X variable, the features
X =df.drop(columns="loan_status")

In [5]:
# Review the y variable Series
y

0        0
1        0
2        0
3        0
4        0
        ..
77531    1
77532    1
77533    1
77534    1
77535    1
Name: loan_status, Length: 77536, dtype: int64

In [6]:
# Review the X variable DataFrame
X.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [7]:
# Check the balance of our target values
y.value_counts

<bound method IndexOpsMixin.value_counts of 0        0
1        0
2        0
3        0
4        0
        ..
77531    1
77532    1
77533    1
77534    1
77535    1
Name: loan_status, Length: 77536, dtype: int64>

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [8]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=1)

---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [9]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
l_model = LogisticRegression(solver='lbfgs', random_state=1)


# Fit the model using training data
l_model.fit(X_train,y_train)

LogisticRegression(random_state=1)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [10]:
# Make a prediction using the testing data
testing_predictions = l_model.predict(X_test)
comparison_df = pd.DataFrame({"Actuals":y_test,
                             "Predictions":testing_predictions})
comparison_df.head()

,Actuals,Predictions
60914,0,0
36843,0,0
1966,0,0
70137,0,0
27237,0,0


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [11]:
# Print the balanced_accuracy score of the model
balanced_acc_score = balanced_accuracy_score(y_test,testing_predictions)
print(f'The balanced accuracy score for this supervised learning model is {balanced_acc_score}')

The balanced accuracy score for this supervised learning model is 0.9520479254722232


In [12]:
# Generate a confusion matrix for the model
con_matrix = confusion_matrix(y_test,testing_predictions)
con_matrix

array([[18663,   102],
       [   56,   563]], dtype=int64)

In [13]:
# Print the classification report for the model
class_report = classification_report(y_test,testing_predictions)
print(class_report)

              precision    recall  f1-score   support

           0       1.00      0.99      1.00     18765
           1       0.85      0.91      0.88       619

    accuracy                           0.99     19384
   macro avg       0.92      0.95      0.94     19384
weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:The precision shows here that every time a loan is flagged healthy, the model is correct. So they could be sure that loans marked as healthy are actually healthy. The only issue is that 15% of the loans marked as risky were actually healthy loans, as seen in the .85 precision for "1" flagging in the classification report. The raw data contains 77,536 loans with 75,036 loans as actually healthy. That is  96.8% of them being healthy loans. Same in the testing data (25% of the total data). Out of 19,384 loans 18,765 are actually healthy. That means by stating all loans are healthy, you would classify get 3.2% of the loans classified as healthy when they are not, by mass approving. The model shows a recall of .99, which means of the healthy loans, it classified them as healthy 99% of the time. For risky loans, it only identified 91% of them as risky, which is relatively not good, when comparing just mass approving, then again there are only 619 loans that are risky in the report meaning, the model would've missed flagging 55.7 of the loans as risky, out of 19384, which is less than 1%. Therefore the model is better than mass approving, where you miss flagging 619 vs 55.7 loans as risky.

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points. 

In [26]:
#had to install imblearn in my conda environment, as I've never used it in this cohort

In [27]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
ros = RandomOverSampler(random_state=1)

# Fit the original training data to the random_oversampler model
X_ros, y_ros =ros.fit_resample(X_train,y_train)

In [28]:
# Count the distinct values of the resampled labels data
y_ros.value_counts()

0    56271
1    56271
Name: loan_status, dtype: int64

### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [30]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
log_ros = LogisticRegression(solver='lbfgs', random_state=1)

# Fit the model using the resampled training data
log_ros.fit(X_ros,y_ros)
# Make a prediction using the testing data
ros_predictions = log_ros.predict(X_test)
ros_predictions

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [33]:
# Print the balanced_accuracy score of the model 
ros_accuracy = balanced_accuracy_score(y_test,ros_predictions)

print(ros_accuracy)

0.9936781215845847


In [37]:
# Generate a confusion matrix for the model
ros_matrix = confusion_matrix(y_test, ros_predictions)


print(ros_matrix)

[[18649   116]
 [    4   615]]


In [39]:
# Print the classification report for the model
ros_class_report = classification_report(y_test,ros_predictions)
print(ros_class_report)

              precision    recall  f1-score   support

           0       1.00      0.99      1.00     18765
           1       0.84      0.99      0.91       619

    accuracy                           0.99     19384
   macro avg       0.92      0.99      0.95     19384
weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:The logistical regression model does worse when it comes to precision on risky loans. As it is down to .84 from .85 precision. Then again it is much better in that it captured almost all of the risky loans as recall is .99 for risky(1) loans. The downside is that it is less accurate (lower) precision but it would be good in the scenario where a company was looking to offer more products and didnt want to offer "bad" candidates any more offers. Downside is that some people with healthy loans would be categorized in the bucket of risky, hence the .84 precision. Lastly, for healthy loans, this model is excellent, as the precision is 100% and recall is .99, meaning the model caught almost all of the healthy loans, with minimal bad loans lumped in there.
